In [240]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fuzzywuzzy # formator data
from fuzzywuzzy import process
from sklearn.impute import SimpleImputer # to change nan to mean value
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv


# Setup
Importing data

In [241]:
vehicle_location = '../input/vehicle-sales-data/car_prices.csv'
vehicle_data = pd.read_csv(vehicle_location)

# Showing table top 5 rows

In [242]:
vehicle_data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


# Droping useless columns

In [243]:
vehicle_data_formated = vehicle_data
vehicle_data_formated.drop(labels = ['interior','seller','trim','saledate','mmr','state'],axis = 1,inplace = True)

# Formating database

## Cleaning transmission

In [244]:
#function to find transmission in columns 
def find_transmission(row):
    for record in row:
        if record == 'automatic' or record == 'manual':
            return record
    return row['transmission']
#apply function and changing data
vehicle_data_formated['transmission'] = vehicle_data.apply(find_transmission, axis=1)
#checking what left
wrong_transmision = vehicle_data_formated[(vehicle_data_formated['transmission'] != 'automatic') & (vehicle_data_formated['transmission'] != 'manual') & (
vehicle_data_formated['transmission'].isna() !=True)]
wrong_transmision

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
461612,2015,Volkswagen,Jetta,Navitgation,Sedan,NaN,NaN,46.0,2711,14250.0
505299,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,36.0,20379,13600.0
529009,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,2817,13750.0
551222,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,9562,13200.0


In [245]:
# i'll delete it cuz there is a lot of missing data
# creating index_transmision_to_delete
index_transmision_to_delete = wrong_transmision.index
#deleting rows
vehicle_data_formated.drop(index_transmision_to_delete, inplace=True)

In [275]:
# showing nan in transmision column
vehicle_data_formated[(vehicle_data_formated.transmission.isna())]

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
70510,2015,audi,NaN,sedan,NaN,waubfgff7f1009810,45.0,3954.0,gray,27500.0
201220,2014,bmw,NaN,sedan,NaN,wbayf8c57ed142026,45.0,34106.0,black,61200.0
410045,2015,audi,NaN,sedan,NaN,waubfgffxf1002043,41.0,4299.0,black,28000.0
475038,2015,audi,NaN,sedan,NaN,waubfgff0f1007039,49.0,8109.0,gray,26900.0
520689,2015,bmw,NaN,sedan,NaN,wba6b4c57fd761030,49.0,7053.0,NaN,88500.0
520731,2015,audi,NaN,sedan,NaN,waubfgff1f1002738,45.0,5271.0,black,26750.0


In [274]:
# taking all unique value 
models_arr = vehicle_data_formated.model.unique()
# main model
def transmission_nan_changer(df:pd.DataFrame):
    for model_name in models_arr:
        transmission_type = df[(df.model == model_name) & (df.transmission.isna() == False)].transmission.unique()
        indexes = df[(df.model == model_name) & (df.transmission.isna() != False)].index
        if len(transmission_type) != 0:
            df.loc[indexes,'transmission'] = transmission_type[0]
        else:
            df.loc[indexes,'transmission'] = 'manual'
    return df
transmission_nan_changer(vehicle_data_formated)
vehicle_data_formated.transmission.unique()


array(['automatic', 'manual', nan], dtype=object)

## Cleaning vin

In [248]:
def find_vin(row):
    for record in row:
        try:
            if len(record) == 17:
                return record
        except:
            pass
    return row['vin']
vehicle_data_formated['vin'] = vehicle_data.apply(find_vin, axis = 1)

In [249]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
188047,2014,Kia,Optima,Sedan,NaN,5xxgm4a79eg329842,35.0,6944.0,white,15300.0
436809,2012,MINI,Cooper,Hatchback,automatic,wmwsv3c52cty28697,27.0,10267.0,blue,14750.0
448327,2009,Chevrolet,Impala,Sedan,NaN,2g1wb57k891107221,36.0,106199.0,black,5000.0
99309,2010,Honda,CR-V,SUV,automatic,5j6re4h73al005575,36.0,66960.0,green,16000.0
535867,2013,Nissan,Altima,sedan,automatic,1n4bl3ap7dc202843,19.0,7142.0,silver,14800.0


In [250]:
# Checking what left
left_vin =  vehicle_data_formated[vehicle_data_formated.vin.str.len() != 17]
left_vin

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
408161,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,4802,13200.0
417835,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,1.0,9410,13300.0
421289,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,1167,13200.0
424161,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,1.0,2172,14050.0
427040,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,41.0,14872,13700.0
427043,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,49.0,12655,13850.0
434424,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,46.0,15719,13650.0
444501,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,2.0,6388,13850.0
453794,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,44.0,16633,13600.0
461597,2015,Volkswagen,Jetta,Navitgation,automatic,automatic,NaN,44.0,11034,13900.0


In [251]:
# Deleting whats left 
vehicle_data_formated.drop(left_vin.index, inplace = True)

## Cleaning car makers

In [252]:
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'].isnull() == False]
car_makers = vehicle_data_formated.make.unique()
np.unique(np.array([car.lower() for car in car_makers]))

array(['acura', 'airstream', 'aston martin', 'audi', 'bentley', 'bmw',
       'buick', 'cadillac', 'chev truck', 'chevrolet', 'chrysler',
       'daewoo', 'dodge', 'dodge tk', 'dot', 'ferrari', 'fiat', 'fisker',
       'ford', 'ford tk', 'ford truck', 'geo', 'gmc', 'gmc truck',
       'honda', 'hummer', 'hyundai', 'hyundai tk', 'infiniti', 'isuzu',
       'jaguar', 'jeep', 'kia', 'lamborghini', 'land rover', 'landrover',
       'lexus', 'lincoln', 'lotus', 'maserati', 'mazda', 'mazda tk',
       'mercedes', 'mercedes-b', 'mercedes-benz', 'mercury', 'mini',
       'mitsubishi', 'nissan', 'oldsmobile', 'plymouth', 'pontiac',
       'porsche', 'ram', 'rolls-royce', 'saab', 'saturn', 'scion',
       'smart', 'subaru', 'suzuki', 'tesla', 'toyota', 'volkswagen',
       'volvo', 'vw'], dtype='<U13')

### Cleaning car makers 

In [253]:
def formating_car_makers(brand):
    if isinstance(brand, str):
        mapping = {
            'chev truck': 'chevrolet',
            'dodge tk': 'dodge',
            'gmc truck': 'gmc',
            'hyundai tk': 'hyundai',
            'land rover': 'landrover',
            'mazda tk': 'mazda',
            'mercedes-b': 'mercedes',
            'mercedes-benz': 'mercedes',
            'vw': 'volkswagen'
        }
        brand = brand.lower()
        if brand in mapping:
            return mapping[brand]
        else:
            return brand
    else:
        ''
    
vehicle_data_formated.make = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'] != 'dot']

/tmp/ipykernel_33/1626176101.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_data_formated.make = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))


## Cleaning car models

In [254]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
381450,2012,nissan,Maxima,Sedan,NaN,1n4aa5ap1cc833759,37.0,39611.0,white,20400.0
446814,2012,infiniti,QX,SUV,NaN,jn8az2nc0c9315166,28.0,40626.0,black,41000.0
119789,2015,kia,Sorento,SUV,automatic,5xyktca68fg584588,5.0,13946.0,white,20800.0
496251,2008,jeep,Patriot,suv,automatic,1j8ft48w68d703407,19.0,143605.0,beige,3600.0
463614,2012,infiniti,G Sedan,g sedan,automatic,jn1dv6ap2cm701009,31.0,34948.0,silver,15500.0


In [255]:
vehicle_data_formated.model = vehicle_data_formated.model.str.lower()
vehicle_data_formated.model = vehicle_data_formated.model.str.strip()


## Cleaning body

In [256]:
vehicle_data_formated.body = vehicle_data_formated.body.str.lower()
vehicle_data_formated.body = vehicle_data_formated.body.str.strip()


In [257]:
vehicle_data_formated.transmission.unique()

array(['automatic', nan, 'manual'], dtype=object)